In [1]:
! nvidia-smi

Thu May 13 06:56:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
print("\"hello\"")
print('"hello"')

"hello"
"hello"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import libraries and pretrained-models

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.1MB 5.1MB/s 
     |████████████████████████████████| 901kB 21.4MB/s 
     |████████████████████████████████| 3.3MB 33.0MB/s 


# Configurations

In [ ]:
import transformers

MAX_LEN = 128
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 8
EPOCHS = 3
BASE_MODEL_PATH = "vinai/phobert-base"
MODEL_PATH = "/content/drive/MyDrive/NER/model.bin"
TRAINING_FILE = "/content/drive/MyDrive/NER/dataset/ner_vcc_train.csv"
TESTING_FILE = "/content/drive/MyDrive/NER/dataset/ner_vcc_test.csv"
TOKENIZER = transformers.AutoTokenizer.from_pretrained(BASE_MODEL_PATH, do_lower_case=True)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
text = "Hôm_nay trời nóng quá nên tôi ở nhà viết Viblo!"
tokens = TOKENIZER.encode(text, add_special_tokens=False)
print('tokens list : ', tokens)
TOKENIZER.decode(tokens)

tokens list :  [3791, 1027, 898, 204, 77, 70, 25, 69, 467, 3696, 16856, 381]


'Hôm_nay trời nóng quá nên tôi ở nhà viết Viblo!'

In [ ]:
text = "hôm_nay trời nóng quá nên tôi ở nhà viết viblo!"
tokens = TOKENIZER.encode(text, add_special_tokens=False)
print('tokens list : ', tokens)
TOKENIZER.decode(tokens)

tokens list :  [1113, 1027, 898, 204, 77, 70, 25, 69, 467, 4104, 16856, 381]


'hôm_nay trời nóng quá nên tôi ở nhà viết viblo!'

In [ ]:
import pandas as pd

train_data = pd.read_csv(TRAINING_FILE)
train_data.head()

,Sentence #,Word,Tag
0,Sentence: 1,Yuval_Noah_Harari,B-PER
1,NaN,nhận,O
2,NaN,bằng,O
3,NaN,Tiến_sĩ,O
4,NaN,của,O


In [ ]:
test_data = pd.read_csv(TESTING_FILE, header=None)
test_data.head()

,0,1,2
0,Sentence #,Word,Tag
1,Sentence: 1,Trong,O
2,NaN,năm,B-TIM
3,NaN,2019,I-TIM
4,NaN,",",O


# Dataset

In [ ]:
import torch

class EntityDataset:
    def __init__(self, texts, tags):
        self.texts = texts
        self.tags = tags
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = self.texts[item]
        tags = self.tags[item]

        ids = []
        target_tag = []

        for i, s in enumerate(text):
            inputs = TOKENIZER.encode(s, add_special_tokens=False)
            input_len = len(inputs)
            ids.extend(inputs)
            target_tag.extend([tags[i]] * input_len)

        ids = ids[:MAX_LEN - 2]
        target_tag = target_tag[:MAX_LEN - 2]

        ids = [0] + ids + [2] # <s>: 0, </s>: 2, <pad>: 1
        target_tag = [12] + target_tag + [12] # O: 12

        mask = [1] * len(ids) # 1: not masked, 0: masked
        token_type_ids = [0] * len(ids)

        padding_len = MAX_LEN - len(ids)

        ids = ids + ([1] * padding_len) # padding
        mask = mask + ([0] * padding_len) # 1: not masked, 0: masked
        token_type_ids = token_type_ids + ([0] * padding_len)
        target_tag = target_tag + ([12] * padding_len) # O: 12

        return {
            "ids": torch.tensor(ids, dtype=torch.long),
            "mask": torch.tensor(mask, dtype=torch.long),
            "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            "target_tag": torch.tensor(target_tag, dtype=torch.long),
        }

# Model

In [ ]:
def loss_fn(output, target, mask, num_labels):
    lfn = torch.nn.CrossEntropyLoss()
    active_loss = mask.view(-1) == 1
    active_logits = output.view(-1, num_labels)
    active_labels = torch.where(
        active_loss,
        target.view(-1),
        torch.tensor(lfn.ignore_index).type_as(target)
    )
    loss = lfn(active_logits, active_labels)
    return loss


class EntityModel(torch.nn.Module):
    def __init__(self, num_tag):
        super(EntityModel, self).__init__()
        self.num_tag = num_tag
        self.bert = transformers.AutoModel.from_pretrained(BASE_MODEL_PATH,return_dict=False)
        # unfreeze bert
        for param in self.bert.parameters():
            param.requires_grad = False
        self.bert_drop = torch.nn.Dropout(0.3)
        self.out_tag = torch.nn.Linear(768, self.num_tag)
    
    def forward(self, ids, mask, token_type_ids, target_tag):
        o, _ = self.bert(ids, attention_mask=mask, token_type_ids=token_type_ids)
        bo_tag = self.bert_drop(o)
        tag = self.out_tag(bo_tag)
        loss = loss_fn(tag, target_tag, mask, self.num_tag)
        return tag, loss

# Engine

In [ ]:
from tqdm import tqdm

def train_fn(data_loader, model, optimizer, device, scheduler):
    model.train()
    final_loss = 0
    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        optimizer.zero_grad()
        _, loss = model(**data)
        loss.backward()
        optimizer.step()
        scheduler.step()
        final_loss += loss.item()
    return final_loss / len(data_loader)


def eval_fn(data_loader, model, device):
    model.eval()
    final_loss = 0
    for data in tqdm(data_loader, total=len(data_loader)):
        for k, v in data.items():
            data[k] = v.to(device)
        _, loss = model(**data)
        final_loss += loss.item()
    return final_loss / len(data_loader)

# Train

In [ ]:
import pandas as pd
import numpy as np

import joblib
import torch

from sklearn import preprocessing
from sklearn import model_selection

from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import KFold

In [ ]:
def process_data(data_path):
    df = pd.read_csv(data_path, encoding="utf-8")
    df.loc[:, "Sentence #"] = df["Sentence #"].fillna(method="ffill")

    enc_tag = preprocessing.LabelEncoder()
    df.loc[:, "Tag"] = enc_tag.fit_transform(df["Tag"])

    sentences = df.groupby("Sentence #")["Word"].apply(list).values
    tag = df.groupby("Sentence #")["Tag"].apply(list).values
    return sentences, tag, enc_tag

In [ ]:
test_sentences, test_tag, _ = process_data(TESTING_FILE)

In [ ]:
enc_tag.classes_

In [ ]:
if __name__ == "__main__":
  train_sentences, train_tag, enc_tag = process_data(TRAINING_FILE)
  test_sentences, test_tag, enc_tag = process_data(TESTING_FILE)
  sentences = np.hstack((train_sentences, test_sentences))
  tag = np.hstack((train_tag, test_tag))  

  # meta_data = {"enc_tag": enc_tag}

  # joblib.dump(meta_data, "/content/drive/MyDrive/NER/meta.bin")

  num_tag = len(list(enc_tag.classes_))
  # train_sentences, test_sentences, train_tag, test_tag = model_selection.train_test_split(sentences, tag, random_state=42, test_size=0.1)

  kf = KFold(n_splits=5)
  for train, test in kf.split(sentences):
    train_sentences, test_sentences, train_tag, test_tag = sentences[train], sentences[test], tag[train], tag[test]

    train_dataset = EntityDataset(texts=train_sentences, tags=train_tag)
    train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)
    test_dataset = EntityDataset(texts=test_sentences, tags=test_tag)
    test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=VALID_BATCH_SIZE, num_workers=1)

    device = torch.device("cuda")
    model = EntityModel(num_tag=num_tag)
    model.load_state_dict(torch.load(MODEL_PATH))
    model.to(device)
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
          {
              "params": [
                  p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
              ],
              "weight_decay": 0.001,
          },
          {
              "params": [
                  p for n, p in param_optimizer if any(nd in n for nd in no_decay)
              ],
              "weight_decay": 0.0,
          },
      ]

    num_train_steps = int(len(train_sentences) / TRAIN_BATCH_SIZE * EPOCHS)
    optimizer = AdamW(optimizer_parameters, lr=3e-5)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)

    epoch_id = 1
    best_loss = np.inf
    for epoch in range(EPOCHS):
      train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
      test_loss = eval_fn(test_data_loader, model, device)
      print(f"Epoch {epoch_id}: Train Loss = {train_loss} Valid Loss = {test_loss}")
      if test_loss < best_loss:
        torch.save(model.state_dict(), MODEL_PATH)
        best_loss = test_loss
      epoch_id += 1

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Test

In [ ]:
!pip install vncorenlp

     |████████████████████████████████| 2.7MB 22.3MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-cp37-none-any.whl size=2645936 sha256=a4580cf417c02ea70ac8f0ba5a9c1efc157c7483a5fc1e76eefdd41caa3f11d7
  Stored in directory: /root/.cache/pip/wheels/09/54/8b/043667de6091d06a381d7745f44174504a9a4a56ecc9380c54
Successfully built vncorenlp


In [ ]:
import joblib
from vncorenlp import VnCoreNLP
import transformers

VNCORENLP_PATH = "/content/drive/MyDrive/NER/VnCoreNLP-1.1.1/VnCoreNLP-1.1.1.jar"
from vncorenlp import VnCoreNLP

segmenter = VnCoreNLP(VNCORENLP_PATH, annotators="wseg", max_heap_size='-Xmx500m')

In [ ]:
meta_data = joblib.load("/content/drive/MyDrive/NER/meta.bin")
enc_tag = meta_data['enc_tag']
enc_tag.classes_

array(['B-EVE', 'B-LOC', 'B-ORG', 'B-PER', 'B-PRO', 'B-TIM', 'I-EVE',
       'I-LOC', 'I-ORG', 'I-PER', 'I-PRO', 'I-TIM', 'O'], dtype=object)

In [ ]:
sentence = "Yuval Noah Harari nhận bằng Tiến sĩ của Đại học Oxford năm 2002, hiện là giảng viên Khoa lịch sử, Đại học Hebrew ở Jerusalem."
segmented_sentence = segmenter.tokenize(sentence)

test_dataset = EntityDataset(texts=[segmented_sentence[0]], tags=[[0] * len(segmented_sentence[0])])

num_tag = len(list(enc_tag.classes_))

model = EntityModel(num_tag=num_tag)
model.load_state_dict(torch.load(MODEL_PATH))
model.to(device)

tags = []
with torch.no_grad():
  data = test_dataset[0]
  for k, v in data.items():
    data[k] = v.to(device).unsqueeze(0)
  tag, _ = model(**data)

decoded_sentence = []
for tokenized_word in test_dataset[0]["ids"]:
  decoded_word = TOKENIZER.decode([tokenized_word])
  decoded_sentence.append(decoded_word)

tags = enc_tag.inverse_transform(tag.argmax(2).cpu().numpy().reshape(-1))[:len(decoded_sentence)]

print("{:15} {:5}".format("Word", "Tag"))
print("="*30)
for w, t in zip(decoded_sentence, tags):
  if w == "<pad>": break
  print("{:15}:{:5}".format(w, t))

Word            Tag  
<s>            :O    
Yu@@           :O    
v@@            :O    
al_@@          :O    
No@@           :B-PER
ah_@@          :O    
Har@@          :O    
ari            :O    
nhận           :O    
bằng           :O    
Tiến_sĩ        :O    
của            :O    
Đại_học        :O    
Oxford         :O    
năm            :O    
2002           :O    
,              :O    
hiện           :O    
là             :O    
giảng_viên     :O    
Khoa           :O    
lịch_sử        :O    
,              :O    
Đại_học        :O    
Heb@@          :B-PER
re@@           :O    
w              :O    
ở              :O    
Jerusalem      :B-LOC
.              :O    
</s>           :O    


In [ ]:
TESTING_FILE = "/content/drive/MyDrive/NER/dataset/ner_vcc_test.csv"
sentences, tag, enc_tag_test = process_data(TESTING_FILE)
test_dataset = EntityDataset(texts=sentences, tags=tag)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=TRAIN_BATCH_SIZE, num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
pred_tag = []
with torch.no_grad():
  for data in test_dataset:
    for k, v in data.items():
      data[k] = v.to(device).unsqueeze(0)
    tag, _ = model(**data)
    pred_tag.append(tag)

predict_tag = []
for tag, sentence in zip(pred_tag, sentences):
  tags = enc_tag.inverse_transform(tag.argmax(2).cpu().numpy().reshape(-1))[:len(sentence) + 2]
  predict_tag.append(tags)

In [ ]:
target_tag = []

for dataset, sentence in zip(test_dataset, sentences):
  tag = dataset['target_tag'][:len(sentence) + 2]
  tag = enc_tag.inverse_transform(tag)
  target_tag.append(tag)

In [ ]:
!pip install sklearn_crfsuite

     |████████████████████████████████| 747kB 6.3MB/s 


In [ ]:
from sklearn_crfsuite import metrics

print(metrics.flat_classification_report(target_tag, predict_tag, labels=enc_tag.classes_[:-1], digits=3))

              precision    recall  f1-score   support

       B-EVE      0.954     0.980     0.967      3883
       B-LOC      0.972     0.984     0.978     24222
       B-ORG      0.974     0.971     0.972     22541
       B-PER      0.984     0.990     0.987     40846
       B-PRO      0.960     0.974     0.967     20955
       B-TIM      0.981     0.988     0.985      8025
       I-EVE      0.965     0.978     0.972      6955
       I-LOC      0.973     0.988     0.981     10421
       I-ORG      0.988     0.978     0.983     18587
       I-PER      0.967     0.971     0.969      2518
       I-PRO      0.974     0.984     0.979     24547
       I-TIM      0.990     0.994     0.992      6863

   micro avg      0.976     0.982     0.979    190363
   macro avg      0.974     0.982     0.978    190363
weighted avg      0.976     0.982     0.979    190363

